
## Upload YouTube Videos to the Reading Material Folder Manually


In this notebook, we will take URLs of videos, convert them to mp3s, upload them to Google Drive, trash them, and delete them.


We will start by ensuring that all the necessary python libraries are installed.

In [ ]:

import sys

print('conda update --all --yes --prefix "{}"'.format(sys.prefix))

In [2]:

import sys

# youtube-dl-2019.8.2
print('{} -m pip install --upgrade youtube_dl'.format(sys.executable))
!{sys.executable} -m pip install --upgrade youtube_dl

C:\Users\dev\anaconda3\python.exe -m pip install --upgrade youtube_dl


In [ ]:

import os

# https://github.com/ytdl-org/youtube-dl/issues/20758
# Just replaced 'token' to 'account_playback_token' in line 1674 of extractor/youtube.py
# All seems to work.
text_editor_path = r'C:\Program Files\Notepad++\notepad++.exe'
#text_editor_path = r'C:\Program Files\Sublime Text 3\sublime_text.exe'
for sub_dir, dirs_list, files_list in os.walk(r'{}\Lib\site-packages\youtube_dl'.format(sys.prefix)):
    for file_name in files_list:
        if 'youtube.py' == file_name:
            file_path = os.path.join(sub_dir, file_name)
            print(file_path)
!"{text_editor_path}" "{file_path}"

In [ ]:

!youtube-dl --version

In [ ]:

!{sys.executable} -m pip install --upgrade PyDrive


Next, we will prove to ourselves that we have ffmpeg correctly installed.

In [ ]:

# https://github.com/adaptlearning/adapt_authoring/wiki/Installing-FFmpeg
!ffmpeg -version


Performing authentication with Google Drive requires we keep our client id and consumer secret in a json file. Keep your secrets off the internet!

In [ ]:

!start %windir%\explorer.exe "C:\Users\dev\Documents\repositories\notebooks\Miscellaneous\json"


We also make sure line 286 of pydrive's files.py file reads <code>self.content.close()</code> before we run the trash cell.

In [ ]:

import sys

file_path = r'{}\Lib\site-packages\pydrive\files.py'.format(sys.prefix)
!"{text_editor_path}" "{file_path}"


Before we start the download process, let's go over setting a few variables:

* **The downloads folder:** This is where all the messy work will be done. All mp3 files and any intermediate files will be deleted out of this folder by the end.
* **The URL list:** This is where we place our YouTube video URLs. It doesn't matter if you remove the <code>time_continue=</code> parameter in the URL, the mp3 will just start at zero seconds anyway.
* **The target folder ID:** This is the folder in Google Drive where our uploads will end up.

In [ ]:

from bs4 import BeautifulSoup as bs
import re
import requests

site_url = 'https://www.youtube.com/'
youtube_css = '#items'
site_page = requests.get(url=site_url)
site_html = site_page.content
site_soup = bs(site_html, 'lxml')
site_soup.select(youtube_css)

In [3]:

import youtube_dl

downloads_folder = r'C:\Users\dev\Downloads'
youtube_url_list = ['https://www.youtube.com/watch?v=_fVnMj_hvD0', 'https://www.youtube.com/watch?v=m9AVo3jXjN8',
                    'https://www.youtube.com/watch?v=OdAYynA83-4', 'https://www.youtube.com/watch?v=PKrq8xqO-yA',
                    'https://www.youtube.com/watch?v=FPKQU3JrDT0', 'https://www.youtube.com/watch?v=NXpWIephX1o',
                    'https://www.youtube.com/watch?v=VkKnOEQlwl4', 'https://www.youtube.com/watch?v=4t9q79wYWAU',
                    'https://www.youtube.com/watch?v=_DBkN5_LQJQ', 'https://www.youtube.com/watch?v=Oohg3LZd898',
                    'https://www.youtube.com/watch?v=80PSGLonjUs', 'https://www.youtube.com/watch?v=GkKJCOSVERg',
                    'https://www.youtube.com/watch?v=EQjBZVssr1I', 'https://www.youtube.com/watch?v=-MDJY5kwm70',
                    'https://www.youtube.com/watch?v=ztYtVVO2orQ', 'https://www.youtube.com/watch?v=Glgcl9AVWbA',
                    'https://www.youtube.com/watch?v=PfAumoTIeik', 'https://www.youtube.com/watch?v=QRkXzv89GPw',
                    'https://www.youtube.com/watch?v=I-hdgbmMryE', 'https://www.youtube.com/watch?v=vG_l7Uz9er8',
                    'https://www.youtube.com/watch?v=96u-_7oApr4', 'https://www.youtube.com/watch?v=ETcq7qqPhow',
                    'https://www.youtube.com/watch?v=exAftDoFAyQ', 'https://www.youtube.com/watch?v=XzGqea4pSgA',
                    'https://www.youtube.com/watch?v=aKQjpvFeolk', 'https://www.youtube.com/watch?v=bzCM__Z12Jg',
                    'https://www.youtube.com/watch?v=dQvSwf72_kI', 'https://www.youtube.com/watch?v=GWk-ZpJdRFg',
                    'https://www.youtube.com/watch?v=1sWdBo-k2iA', 'https://www.youtube.com/watch?v=PMarFp_-0T0',
                    'https://www.youtube.com/watch?v=UA-YaJYWRjs', 'https://www.youtube.com/watch?v=rnOnkHxR7EM',
                    'https://www.youtube.com/watch?v=hqn5ZLxvRSU', 'https://www.youtube.com/watch?v=ctltnYu2hgo',
                    'https://www.youtube.com/watch?v=KiedF6T-Tw0', 'https://www.youtube.com/watch?v=D0NM3V3QQrE',
                    'https://www.youtube.com/watch?v=4lJlxs8G9z4', 'https://www.youtube.com/watch?v=4kJQAXugOaY',
                    'https://www.youtube.com/watch?v=R5Z_JaXep3M', 'https://www.youtube.com/watch?v=9oIpb4zYxt4',
                    'https://www.youtube.com/watch?v=ggrW6qi0qZQ', 'https://www.youtube.com/watch?v=z3LDEN5TLaQ',
                    'https://www.youtube.com/watch?v=zA0VcREGdQ0', ]

Now we are ready to start the download process by running <code>youtube_dl</code> on our <code>youtube_url_list</code>.

In [4]:

import os

ydl_opts = {
    'format': 'bestaudio/best',
    'nocheckcertificate': False,
    'outtmpl': os.path.join(downloads_folder, youtube_dl.DEFAULT_OUTTMPL),
    'postprocessors': [{'key': 'FFmpegExtractAudio',
                       'preferredcodec': 'mp3',
                       'preferredquality': '192'}],
    'verbose': True,
    }
with youtube_dl.YoutubeDL(ydl_opts) as ydl:
    for url in youtube_url_list:
        try:
            # It also downloads the videos
            res = ydl.extract_info(url,
                                   force_generic_extractor=ydl.params.get('force_generic_extractor',
                                                                          False))
        except Exception as e:
            print()
            print('####################################################')
            print('{}: {}'.format(url, e))
            print('####################################################')
            print()

    print(ydl._download_retcode)
print('Conversion completed.')

[debug] Encodings: locale cp1252, fs utf-8, out UTF-8, pref cp1252
[debug] youtube-dl version 2020.06.16.1
[debug] Python version 3.7.6 (CPython) - Windows-10-10.0.18362-SP0
[debug] exe versions: avconv v13_dev0-1440-g34c1133, avprobe v13_dev0-1440-g34c1133, ffmpeg git-2020-01-08-5bd0010, ffprobe git-2020-01-08-5bd0010
[debug] Proxy map: {}


[youtube] _fVnMj_hvD0: Downloading webpage
[debug] Invoking downloader on 'https://r8---sn-bvvbax-cvns.googlevideo.com/videoplayback?expire=1592718195&ei=E5_uXp3vJouwhgbupZT4Dg&ip=24.91.86.222&id=o-AE9tInn2zVHILETVjWl3kLLDG7V0TcnA7AgVelKQh-uh&itag=140&source=youtube&requiressl=yes&mh=Ca&mm=31%2C29&mn=sn-bvvbax-cvns%2Csn-ab5l6ndr&ms=au%2Crdu&mv=m&mvi=7&pl=21&initcwndbps=1917500&vprv=1&mime=audio%2Fmp4&gir=yes&clen=9882135&dur=610.568&lmt=1592598282378072&mt=1592696549&fvip=5&keepalive=yes&c=WEB&txp=6311222&sparams=expire%2Cei%2Cip%2Cid%2Citag%2Csource%2Crequiressl%2Cvprv%2Cmime%2Cgir%2Cclen%2Cdur%2Clmt&sig=AOq0QJ8wRAIgT_l3uTiMNqrPHsQytHYVUqicsduBQ6yXHGLou6ootqgCIFU3oUOLXKPawH-JQKvhuZwaj1FNAuQQlugVsISFDFHf&lsparams=mh%2Cmm%2Cmn%2Cms%2Cmv%2Cmvi%2Cpl%2Cinitcwndbps&lsig=AG3C_xAwRAIgQdgfV6DNBc4RI4qJqRzRqa7adffHcGcgTtaByui_fToCIA50hakjEKPQ4Z7WojtXo8-3lUmYNEmIgwutulmW_-oY&ratebypass=yes'
[download] Destination: C:\Users\dev\Downloads\Fractal Salad (Lena 183-227)-_fVnMj_hvD0.m4a
[download] 100%

ERROR: -MDJY5kwm70: YouTube said: This video has been removed by the uploader
Sorry about that.
Traceback (most recent call last):
  File "C:\Users\dev\anaconda3\lib\site-packages\youtube_dl\YoutubeDL.py", line 797, in extract_info
    ie_result = ie.extract(url)
  File "C:\Users\dev\anaconda3\lib\site-packages\youtube_dl\extractor\common.py", line 530, in extract
    ie_result = self._real_extract(url)
  File "C:\Users\dev\anaconda3\lib\site-packages\youtube_dl\extractor\youtube.py", line 1893, in _real_extract
    'YouTube said: %s' % unavailable_message, expected=True, video_id=video_id)
youtube_dl.utils.ExtractorError: -MDJY5kwm70: YouTube said: This video has been removed by the uploader
Sorry about that.




####################################################
https://www.youtube.com/watch?v=-MDJY5kwm70: ERROR: -MDJY5kwm70: YouTube said: This video has been removed by the uploader
Sorry about that.
####################################################

[youtube] ztYtVVO2orQ: Downloading webpage
[debug] Invoking downloader on 'https://r2---sn-bvvbax-cvnz.googlevideo.com/videoplayback?expire=1592719844&ei=hKXuXpmFLYynhgbk0oTIAg&ip=24.91.86.222&id=o-AA5QaYYaVhF5jBhdbpJKL9vtfAvaUHzNi21g98Z94Mfg&itag=140&source=youtube&requiressl=yes&mh=OQ&mm=31%2C29&mn=sn-bvvbax-cvnz%2Csn-ab5szn76&ms=au%2Crdu&mv=m&mvi=1&pl=21&initcwndbps=2186250&vprv=1&mime=audio%2Fmp4&gir=yes&clen=42994420&dur=2656.571&lmt=1591738795744537&mt=1592698175&fvip=5&keepalive=yes&c=WEB&txp=5431432&sparams=expire%2Cei%2Cip%2Cid%2Citag%2Csource%2Crequiressl%2Cvprv%2Cmime%2Cgir%2Cclen%2Cdur%2Clmt&sig=AOq0QJ8wRAIgamKkLfw0L248jKY09k3RPlbfJ4DkbwV-7ftGvhWrkTsCICb3TWp_0O3Oxc-Lw4x8rlaopWLLbf2178NHDYhDFMc8&lsparams=mh%2Cmm%2Cmn%2Cms%2Cmv%2Cm

ERROR: Content Warning
If the owner of this video has granted you access, please sign in.
       This video is private.
Traceback (most recent call last):
  File "C:\Users\dev\anaconda3\lib\site-packages\youtube_dl\YoutubeDL.py", line 797, in extract_info
    ie_result = ie.extract(url)
  File "C:\Users\dev\anaconda3\lib\site-packages\youtube_dl\extractor\common.py", line 530, in extract
    ie_result = self._real_extract(url)
  File "C:\Users\dev\anaconda3\lib\site-packages\youtube_dl\extractor\youtube.py", line 2212, in _real_extract
    raise ExtractorError(error_message, expected=True)
youtube_dl.utils.ExtractorError: Content Warning
If the owner of this video has granted you access, please sign in.
       This video is private.




####################################################
https://www.youtube.com/watch?v=96u-_7oApr4: ERROR: Content Warning
If the owner of this video has granted you access, please sign in.
       This video is private.
####################################################

[youtube] ETcq7qqPhow: Downloading webpage
[debug] Invoking downloader on 'https://r2---sn-bvvbax-cvnl.googlevideo.com/videoplayback?expire=1592720744&ei=CKnuXsu_E4fyhgb85LiACg&ip=24.91.86.222&id=o-AOGnivnVIZw3xCWYxj_MvuzhbyKEbZDHwYE7a0nMSqrc&itag=140&source=youtube&requiressl=yes&mh=1e&mm=31%2C29&mn=sn-bvvbax-cvnl%2Csn-ab5l6ndy&ms=au%2Crdu&mv=m&mvi=1&pl=21&initcwndbps=1648750&vprv=1&mime=audio%2Fmp4&gir=yes&clen=9794523&dur=605.158&lmt=1591443659685857&mt=1592699073&fvip=2&keepalive=yes&c=WEB&txp=5431432&sparams=expire%2Cei%2Cip%2Cid%2Citag%2Csource%2Crequiressl%2Cvprv%2Cmime%2Cgir%2Cclen%2Cdur%2Clmt&sig=AOq0QJ8wRgIhAJSVuZxNsM1jXL4exXUZAhkEN0AX_kJa57PnqWnEgVfrAiEAiGu4UY85fANr6sY9GIkqFCyDHIy6cxHYcbVg_Eoe344%3D&lsparam

ERROR: unable to download video data: <urlopen error [Errno 11001] getaddrinfo failed>
Traceback (most recent call last):
  File "C:\Users\dev\anaconda3\lib\urllib\request.py", line 1319, in do_open
    encode_chunked=req.has_header('Transfer-encoding'))
  File "C:\Users\dev\anaconda3\lib\http\client.py", line 1252, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "C:\Users\dev\anaconda3\lib\http\client.py", line 1298, in _send_request
    self.endheaders(body, encode_chunked=encode_chunked)
  File "C:\Users\dev\anaconda3\lib\http\client.py", line 1247, in endheaders
    self._send_output(message_body, encode_chunked=encode_chunked)
  File "C:\Users\dev\anaconda3\lib\http\client.py", line 1026, in _send_output
    self.send(msg)
  File "C:\Users\dev\anaconda3\lib\http\client.py", line 966, in send
    self.connect()
  File "C:\Users\dev\anaconda3\lib\http\client.py", line 1414, in connect
    super().connect()
  File "C:\Users\dev\anaconda3\lib\http\


####################################################
https://www.youtube.com/watch?v=4kJQAXugOaY: ERROR: unable to download video data: <urlopen error [Errno 11001] getaddrinfo failed>
####################################################

[youtube] R5Z_JaXep3M: Downloading webpage
[debug] Invoking downloader on 'https://r7---sn-bvvbax-cvne.googlevideo.com/videoplayback?expire=1592722121&ei=aK7uXtiMNsXNgwPX_Z-4BQ&ip=24.91.86.222&id=o-ADOMlMwIYHyxP7IHjcDuVd3kQ_tJc2hBpfqTR-MQBlmc&itag=140&source=youtube&requiressl=yes&mh=X1&mm=31%2C29&mn=sn-bvvbax-cvne%2Csn-ab5l6nzl&ms=au%2Crdu&mv=m&mvi=6&pl=21&initcwndbps=1370000&vprv=1&mime=audio%2Fmp4&gir=yes&clen=2189312&dur=135.233&lmt=1590581046532259&mt=1592700450&fvip=5&keepalive=yes&c=WEB&txp=5431432&sparams=expire%2Cei%2Cip%2Cid%2Citag%2Csource%2Crequiressl%2Cvprv%2Cmime%2Cgir%2Cclen%2Cdur%2Clmt&sig=AOq0QJ8wRAIgc93Tcw0kZIBjanJ5nEKJxL5j05bTgUm7P3xcrnlMUdsCIBJdUKodsAwtNZIej_8ZLtEKelP3-cdI0fORNOOfbv8U&lsparams=mh%2Cmm%2Cmn%2Cms%2Cmv%2Cmvi%2Cpl%2Ci

In [3]:

!start %windir%\explorer.exe "{os.path.abspath(downloads_folder)}"

In [4]:

import re

files_list = os.listdir(downloads_folder)
for file_name in files_list:
    if file_name.endswith('.mp3'):
        src_path = os.path.join(downloads_folder, file_name)
        file_name = re.sub(r'-[A-Z0-9a-z-_]{11}\.mp3', '.mp3', file_name)
        dst_path = os.path.join(downloads_folder, file_name)
        os.rename(src_path, dst_path)

In [5]:

# ID of the "Reading Material" folder
tgt_folder_id = '1syfUx6jukbW1CWIEy8xoM9veGGr5MBUh'
rm_url = 'https://drive.google.com/drive/u/0/folders/{}'.format(tgt_folder_id)
!"C:\Program Files (x86)\Google\Chrome\Application\chrome.exe" {rm_url}


Next, we authenticate with Google by running the builtin web server conveniently provided.

In [ ]:

from pydrive.auth import GoogleAuth

# The URL that gets displayed here is not suitable for public consumption
GoogleAuth.DEFAULT_SETTINGS['client_config_file'] = r'../json/client_secret.json'
gauth = GoogleAuth()
gauth.LocalWebserverAuth()


Before you run the next cell, in the online web page of Google Drive, bring up the share dialog box of the folder you are uploading to and copy its folder id into the <code>tgt_folder_id</code> string. If you right-click on the folder and select **Share...** and then click **Get shareable link**, you will see something like:

https://drive.google.com/drive/folders/1syfUx6jukbW1CWIEy8xoM9veGGr5MBUx?usp=sharing

It's that **1syfUx6jukbW1CWIEy8xoM9veGGr5MBUx** that you need to copy.

In [ ]:

from pydrive.drive import GoogleDrive

# Create GoogleDrive instance with authenticated GoogleAuth instance
drive = GoogleDrive(gauth)

# ID of the "Reading Material" folder
tgt_folder_id = '1syfUx6jukbW1CWIEy8xoM9veGGr5MBUh'


Next, we will upload all the mp3s in our download folder.

In [ ]:

import os

# Upload all mp3s from the Downloads folder to Google Drive's "Reading Material" folder
gfile_dict = {}
for subdir, dirs, files in os.walk(downloads_folder):
    for src_file in files:
        if src_file.endswith('.mp3'):
            src_path = os.path.join(subdir, src_file)
            title = '-'.join(src_file.split('-')[:-1]) + '.mp3'
            gfile_dict[src_file] = drive.CreateFile({'title':title, 'mimeType':'audio/mp3',
                                                     'parents': [{'kind': 'drive#fileLink',
                                                                  'id': tgt_folder_id}]})

            # Read mp3 file and set it as a content of this instance
            gfile_dict[src_file].SetContentFile(src_path)
            
            # Upload the file
            try:
                gfile_dict[src_file].Upload()
                print('Uploaded %s (%s)' % (gfile_dict[src_file]['title'],
                                            gfile_dict[src_file]['mimeType']))
            except Exception as e:
                print('Upload failed for %s (%s): %s' % (gfile_dict[src_file]['title'],
                                                         gfile_dict[src_file]['mimeType'], e))
print('Upload completed.')


Download the mp3s to <a href='http://www.voicedream.com/support/user-manual/#file'>Voice Dream</a> before you run the trash cell (below).

In [ ]:

import os

# Trash all mp3s from Google Drive's "Reading Material" folder
for src_file in gfile_dict.keys():

    # Trash mp3 file
    try:
        gfile_dict[src_file].Trash()
        print('Trashed %s (%s)' % (gfile_dict[src_file]['title'],
                                   gfile_dict[src_file]['mimeType']))
    except Exception as e:
        print('Trash failed for %s (%s): %s' % (gfile_dict[src_file]['title'],
                                                gfile_dict[src_file]['mimeType'], e))
    
print('Trashing completed.')


Lastly, we run the code that will delete all the mp3s out of our downloads folder.

In [ ]:

import os

# Delete all mp3s in the Downloads folder
for src_file in gfile_dict.keys():
    src_path = os.path.join(downloads_folder, src_file)

    # Delete the file
    try:
        os.remove(src_path)
        print('Deleted %s (%s)' % (src_file,
                                   gfile_dict[src_file]['mimeType']))
    except Exception as e:
        print('Failed to delete %s (%s): %s' % (src_file,
                                                gfile_dict[src_file]['mimeType'], e))
print('Deleting completed.')


If you open the downloads folder with the code below you can see that the deletion is complete.

In [ ]:

!start %windir%\explorer.exe "{os.path.abspath(downloads_folder)}"

In [ ]:

from IPython.display import HTML

rm_url = 'https://drive.google.com/drive/u/0/folders/{}'.format(tgt_folder_id)
message_str = 'open up your Google Drive target folder'
rm_link = '<a href="{}" target="_blank">{}</a>'.format(rm_url, message_str)
message_str = 'If you want to, you can run the code below to {} and'.format(rm_link)
message_str += ' check if everything got deleted.'
HTML(message_str)

In [ ]:

!"C:\Program Files (x86)\Google\Chrome\Application\chrome.exe" {rm_url}